In [123]:
import os
from dotenv import load_dotenv
from bs4 import BeautifulSoup
import requests
import pandas as pd
import requests
import json
from random import randint
from time import sleep

In [3]:
load_dotenv()

True

In [66]:
page = requests.get('https://footballapi.pulselive.com/football/stats/match/38679',headers = {
        "origin": "https://www.premierleague.com"
    }).text
match_details = json.loads(page)

matchweek = match_details['entity']['gameweek']['gameweek']
kickoff = match_details['entity']['kickoff']['label']
home_team = match_details['entity']['teams'][0]['team']['name']
home_result = match_details['entity']['teams'][0]['score']
away_team = match_details['entity']['teams'][1]['team']['name']
away_result = match_details['entity']['teams'][1]['score']
stadium_name = match_details['entity']['ground']['name']
stadium_city = match_details['entity']['ground']['city']
attendance = match_details['entity']['attendance']
clock = match_details['entity']['clock']['secs']

In [116]:
def get_soup(url):
    page = requests.get(url,headers = {"User-Agent":"Mozilla/5.0","origin":"https://www.premierleague.com"})
    soup = BeautifulSoup(page.content, 'html.parser')
    return soup

In [122]:
matchweek_df = pd.DataFrame([],columns=['matchweek',
                                                 'kickoff',
                                                 'stadium_name',
                                                 'stadium_city',
                                                 'attendance',
                                                 'clock',
                                                 'home_team',
                                                 'away_team',
                                                 'home_result',
                                                 'away_result'] )
for season in 210,274,363:
    for page_number in range(0,10):
        page = requests.get(f'''

        https://footballapi.pulselive.com/football/fixtures?comps=1&compSeasons={season}
        &page={page_number}
        &pageSize=40&sort=desc&statuses=C&altIds=true
        ''',headers = {
                "origin": "https://www.premierleague.com"
            }).text
        matchweek_details = json.loads(page)
        for match in matchweek_details['content']:
            matchweek = match['gameweek']['gameweek']
            kickoff = match['kickoff']['label']
            home_team = match['teams'][0]['team']['name']
            home_result = match['teams'][0]['score']
            away_team = match['teams'][1]['team']['name']
            away_result = match['teams'][1]['score']
            stadium_name = match['ground']['name']
            stadium_city = match['ground']['city']
            clock = match['clock']['secs']
            
            match_soup = get_soup('https://www.premierleague.com/match/'+ str(int(match['id'])))
            referee = match_soup.find('div',{'class':'referee'}).text.replace('\n','').strip()

            
                
            match_details_df = pd.DataFrame([[matchweek,
                                          kickoff,
                                              referee,
                                          stadium_name,
                                          stadium_city,
                                          clock,
                                          home_team,
                                          away_team,
                                          home_result,
                                          away_result
                                         ]],columns=['matchweek',
                                                     'kickoff',
                                                     'referee',
                                                     'stadium_name',
                                                     'stadium_city',
                                                     'clock',
                                                     'home_team',
                                                     'away_team',
                                                     'home_result',
                                                     'away_result']
                                       )

            if 'attendance' in match:
                match_details_df['attendance'] = match['attendance']
            else:
                match_details_df['attendance'] = 0
                
            matchweek_df = matchweek_df.append(match_details_df)
matchweek_df.to_csv('../CSVs/matches.csv')
matchweek_df

,matchweek,kickoff,stadium_name,stadium_city,attendance,clock,home_team,away_team,home_result,away_result,referee
0,38.0,"Sun 12 May 2019, 15:00 BST",Amex Stadium,Falmer,30662.0,5580.0,Brighton and Hove Albion,Manchester City,1.0,4.0,Michael Oliver
0,38.0,"Sun 12 May 2019, 15:00 BST",Turf Moor,Burnley,21461.0,5700.0,Burnley,Arsenal,1.0,3.0,Mike Dean
0,38.0,"Sun 12 May 2019, 15:00 BST",Selhurst Park,London,25433.0,5820.0,Crystal Palace,AFC Bournemouth,5.0,3.0,Roger East
0,38.0,"Sun 12 May 2019, 15:00 BST",Craven Cottage,London,24979.0,5580.0,Fulham,Newcastle United,0.0,4.0,Kevin Friend
0,38.0,"Sun 12 May 2019, 15:00 BST",King Power Stadium,Leicester,32140.0,5640.0,Leicester City,Chelsea,0.0,0.0,Anthony Taylor
...,...,...,...,...,...,...,...,...,...,...,...
0,1.0,"Sun 13 Sep 2020, 14:00 BST",The Hawthorns,West Bromwich,0.0,5580.0,West Bromwich Albion,Leicester City,0.0,3.0,Anthony Taylor
0,1.0,"Sat 12 Sep 2020, 20:00 BST",London Stadium,London,0.0,5700.0,West Ham United,Newcastle United,0.0,2.0,Stuart Attwell
0,1.0,"Sat 12 Sep 2020, 17:30 BST",Anfield,Liverpool,0.0,5700.0,Liverpool,Leeds United,4.0,3.0,Michael Oliver
0,1.0,"Sat 12 Sep 2020, 15:00 BST",Selhurst Park,London,0.0,5820.0,Crystal Palace,Southampton,1.0,0.0,Jonathan Moss


In [124]:
stats_df = pd.DataFrame([],columns=['team','stat','value'])
for season in 210,274,363:
    for page_number in range(0,10):
        page = requests.get(f'''

        https://footballapi.pulselive.com/football/fixtures?comps=1&compSeasons={season}
        &page={page_number}
        &pageSize=40&sort=desc&statuses=C&altIds=true
        ''',headers = {
                "origin": "https://www.premierleague.com"
            }).text
        matchweek_details = json.loads(page)
        for match in matchweek_details['content']:
            page = requests.get('https://footballapi.pulselive.com/football/stats/match/'+str(int(match['id'])),headers = {
                "origin": "https://www.premierleague.com"
            }).text
            match_details = json.loads(page)
            
            for team in match_details['entity']['teams']:
                for stat in match_details['data'][str(team['team']['id'])]['M']:
                    stat_df = pd.DataFrame([[team['team']['name'],stat['name'],stat['value']]],columns=['team','stat','value'])
                    stats_df = stats_df.append(stat_df)
            sleep(randint(1,3))
                    
stats_df['matchweek'] = matchweek
stats_df['kickoff'] = kickoff
stats_df
                

stats_df.to_csv('../CSVs/match_stats.csv')
stats_df

,team,stat,value,matchweek,kickoff
0,Brighton and Hove Albion,formation_used,451.000000,1.0,"Sat 12 Sep 2020, 12:30 BST"
0,Brighton and Hove Albion,total_distance_in_m,99424.710938,1.0,"Sat 12 Sep 2020, 12:30 BST"
0,Brighton and Hove Albion,total_back_zone_pass,123.000000,1.0,"Sat 12 Sep 2020, 12:30 BST"
0,Brighton and Hove Albion,total_pass,245.000000,1.0,"Sat 12 Sep 2020, 12:30 BST"
0,Brighton and Hove Albion,backward_pass,49.000000,1.0,"Sat 12 Sep 2020, 12:30 BST"
...,...,...,...,...,...
0,Arsenal,total_through_ball,2.000000,1.0,"Sat 12 Sep 2020, 12:30 BST"
0,Arsenal,accurate_through_ball,2.000000,1.0,"Sat 12 Sep 2020, 12:30 BST"
0,Arsenal,att_goal_high_right,1.000000,1.0,"Sat 12 Sep 2020, 12:30 BST"
0,Arsenal,successful_fifty_fifty,17.000000,1.0,"Sat 12 Sep 2020, 12:30 BST"


In [71]:
referee = match_soup.find('div',{'class':'referee'}).text.replace('\n','').strip()

match_details_df = pd.DataFrame([[matchweek,
                                  kickoff,
                                  referee,
                                  stadium_name,
                                  stadium_city,
                                  attendance,
                                  clock,
                                  home_team,
                                  away_team,
                                  home_result,
                                  away_result
                                 ]],columns=['matchweek',
                                             'kickoff',
                                             'referee',
                                             'stadium_name',
                                             'stadium_city',
                                             'attendance',
                                             'clock',
                                             'home_team',
                                             'away_team',
                                             'home_result',
                                             'away_result']
                               )
match_details_df

,matchweek,kickoff,referee,stadium_name,stadium_city,attendance,clock,home_team,away_team,home_result,away_result
0,38,"Sun 12 May 2019, 15:00 BST",Mike Dean,Turf Moor,Burnley,21461,5700,Burnley,Arsenal,1,3


,team,stat,value,matchweek,kickoff
0,Burnley,formation_used,442.000000,Matchweek 38,"Sun 12 May 2019, 15:00 BST"
0,Burnley,total_distance_in_m,116019.679688,Matchweek 38,"Sun 12 May 2019, 15:00 BST"
0,Burnley,accurate_fwd_zone_pass,162.000000,Matchweek 38,"Sun 12 May 2019, 15:00 BST"
0,Burnley,interception,13.000000,Matchweek 38,"Sun 12 May 2019, 15:00 BST"
0,Burnley,total_flick_on,3.000000,Matchweek 38,"Sun 12 May 2019, 15:00 BST"
...,...,...,...,...,...
0,Arsenal,att_goal_low_centre,1.000000,Matchweek 38,"Sun 12 May 2019, 15:00 BST"
0,Arsenal,att_lf_goal,1.000000,Matchweek 38,"Sun 12 May 2019, 15:00 BST"
0,Arsenal,wins,1.000000,Matchweek 38,"Sun 12 May 2019, 15:00 BST"
0,Arsenal,total_flick_on,3.000000,Matchweek 38,"Sun 12 May 2019, 15:00 BST"


#Testing with Bunrley Vs Arsenal

In [87]:
timeline_div = match_soup.find('div',{'class':'timeLineEventsContainer'})

match_df = pd.DataFrame([],columns=['team','player','event','minute'])
matchweek = match_soup.find('div',{'class':'long'}).text
match_date = match_soup.find('div',{'class':'matchDate'})['data-kickoff']
referee = match_soup.find('div',{'class':'referee'}).text
stadium = match_soup.find('div',{'class':'stadium'}).text
attendance = match_soup.find('div',{'class':'attendance'}).text
kick_off = match_soup.find('strong',{'class':'renderKOContainer'}).text
match_details_df = pd.DataFrame([[matchweek,match_date, referee, stadium, attendance]],columns=['matchweek','match_date','referee','stadium','attendance'])
match_details_df['referee'] = match_details_df['referee'].str.replace('\n','').str.strip()
match_details_df['stadium'] = match_details_df['stadium'].str.replace('\n','').str.strip()
match_details_df['attendance'] = match_details_df['attendance'].str.replace('\n','').str.strip()
match_details_df['match_date'] = pd.to_datetime(match_details_df['match_date'],unit='ms')
for timeline_item in timeline_div.find_all('div',{'class':['home','away']}):
    if 'home' in timeline_item['class']:
        team_name = home_team
    else:
        team_name = away_team
    player = timeline_item.find('a',{'class':'name'}).text
    player = player.split('\n')[0]
    
    event_df = pd.DataFrame([[team_name,player,timeline_item.find('span',{'class':'visuallyHidden'}).text,timeline_item.find('time',{'class':'min'})]],columns=['team','player','event','minute'])
    match_df = match_df.append(event_df)
    
    if timeline_item.find('div',{'class':'subOn'}):
        sub_on = timeline_item.find('div',{'class':'subOn'}).find('a',{'class':'name'}).text
        sub_on = sub_on.split('\n')[0]
        event_df = pd.DataFrame([[team_name,sub_on,'Substitution (on)',timeline_item.find('time',{'class':'min'})]],columns=['team','player','event','minute'])
        match_df = match_df.append(event_df)
match_df['team'] = match_df['team'].str.replace('\n','').str.strip()
match_df[['number','player']] = match_df['player'].str.split('. ',n=1,expand=True)
match_df['matchweek'] = matchweek
match_df['kickoff'] = kickoff
print(match_details_df)
match_df

      matchweek          match_date    referee             stadium  \
0  Matchweek 38 2019-05-12 14:00:00  Mike Dean  Turf Moor, Burnley   

    attendance  
0  Att: 21,461  


,team,player,event,minute,number,matchweek,kickoff
0,Burnley,Tom Heaton,Yellow Card,[3'],1,Matchweek 38,"Sun 12 May 2019, 15:00 BST"
0,Burnley,Ashley Barnes,Yellow Card,[22'],10,Matchweek 38,"Sun 12 May 2019, 15:00 BST"
0,Arsenal,Konstantinos Mavropanos,Substitution,[34'],27,Matchweek 38,"Sun 12 May 2019, 15:00 BST"
0,Arsenal,Laurent Koscielny,Substitution (on),[34'],6,Matchweek 38,"Sun 12 May 2019, 15:00 BST"
0,Burnley,James Tarkowski,Yellow Card,[45 +3'],5,Matchweek 38,"Sun 12 May 2019, 15:00 BST"
0,Burnley,Matthew Lowton,Yellow Card,[48'],2,Matchweek 38,"Sun 12 May 2019, 15:00 BST"
0,Arsenal,Pierre-Emerick Aubameyang,Goal,[52'],14,Matchweek 38,"Sun 12 May 2019, 15:00 BST"
0,Arsenal,Mattéo Guendouzi,Yellow Card,[52'],29,Matchweek 38,"Sun 12 May 2019, 15:00 BST"
0,Arsenal,Joseph Willock,Substitution,[62'],59,Matchweek 38,"Sun 12 May 2019, 15:00 BST"
0,Arsenal,Eddie Nketiah,Substitution (on),[62'],49,Matchweek 38,"Sun 12 May 2019, 15:00 BST"


In [77]:
line_up_div = match_soup.find_all('div',{'class':'matchLineupTeamContainer'})
lineup_df = pd.DataFrame([],columns=['team','player','number','position','captain','substitution'])

for team in line_up_div:
    if team['data-index-class'] == 'home':
        team_name = home_team
    else:
        team_name = away_team
    for index, ul in enumerate(team.find_all('ul',{'class':'startingLineUpContainer'})):
        for player in ul.find_all('li',{'class':'player'}):
            player_lineup_df = pd.DataFrame([[team_name,
                                              player.find('div',{'class':'name'}).contents[0].strip(),
                                              player.find('div',{'class':'number'}).text.strip(),
                                              player.find('span',{'class':'position'}).text.strip(),
                                            index
                                             ]],columns=['team','player','number','position','substitution'])
            if player.find('div',{'class':'cpt'}):
                player_lineup_df['captain'] = 1
                player_lineup_df['position'] = player_lineup_df['position'].str.replace('C\n','')
            else:
                player_lineup_df['captain'] = 0
            lineup_df = lineup_df.append(player_lineup_df)

lineup_df['matchweek'] = matchweek
lineup_df['match_date'] = kickoff
lineup_df

,team,player,number,position,captain,substitution,matchweek,match_date
0,Burnley,Tom Heaton,1,Goalkeeper,1,0,Matchweek 38,"Sun 12 May 2019, 15:00 BST"
0,Burnley,Charlie Taylor,3,Defender,0,0,Matchweek 38,"Sun 12 May 2019, 15:00 BST"
0,Burnley,Ben Mee,6,Defender,0,0,Matchweek 38,"Sun 12 May 2019, 15:00 BST"
0,Burnley,Matthew Lowton,2,Defender,0,0,Matchweek 38,"Sun 12 May 2019, 15:00 BST"
0,Burnley,James Tarkowski,5,Defender,0,0,Matchweek 38,"Sun 12 May 2019, 15:00 BST"
0,Burnley,Ashley Westwood,18,Midfielder,0,0,Matchweek 38,"Sun 12 May 2019, 15:00 BST"
0,Burnley,Jeff Hendrick,13,Midfielder,0,0,Matchweek 38,"Sun 12 May 2019, 15:00 BST"
0,Burnley,Dwight McNeil,31,Midfielder,0,0,Matchweek 38,"Sun 12 May 2019, 15:00 BST"
0,Burnley,Jack Cork,4,Midfielder,0,0,Matchweek 38,"Sun 12 May 2019, 15:00 BST"
0,Burnley,Ashley Barnes,10,Forward,0,0,Matchweek 38,"Sun 12 May 2019, 15:00 BST"
